In [1]:
from tensorflow import keras
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [2]:
base_model.trainable = False

In [3]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation = 'softmax')(x)
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
model.compile(loss=keras.losses.categorical_crossentropy , metrics = ['accuracy'] )

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(samplewise_center=True, rotation_range=10, zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=True, 
        vertical_flip=False)

In [7]:
train_it = datagen.flow_from_directory('data/fruits/train', 
                                       target_size=(224, 224),
                                       color_mode='rgb', 
                                       class_mode="categorical")
valid_it = datagen.flow_from_directory('data/fruits/valid', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


In [8]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=2)

Epoch 1/2
37/36 [==============================] - 27s 731ms/step - loss: 0.5835 - accuracy: 0.9248 - val_loss: 0.2538 - val_accuracy: 0.9802
Epoch 2/2
37/36 [==============================] - 19s 521ms/step - loss: 0.1930 - accuracy: 0.9824 - val_loss: 0.1518 - val_accuracy: 0.9858


In [9]:
base_model.trainable = True

model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss = keras.losses.categorical_crossentropy , metrics = ['accuracy'])

In [10]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=2)

Epoch 1/2
37/36 [==============================] - 33s 883ms/step - loss: 0.0941 - accuracy: 0.9896 - val_loss: 0.0669 - val_accuracy: 0.9959
Epoch 2/2
37/36 [==============================] - 21s 556ms/step - loss: 0.0478 - accuracy: 0.9953 - val_loss: 0.0670 - val_accuracy: 0.9954


In [11]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

11/10 [================================] - 4s 337ms/step - loss: 0.0595 - accuracy: 0.9970


[0.05948620289564133, 0.996960461139679]

In [12]:
model.save("myModel.h5")